In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt

In [2]:
def load_all_pickles(folder_path):
    '''The function load all the pickle files from a given folder. It assumes that the folder only
    contains .pkl files. If different extensions exist it will return an error
    Parameters
    ----------
    folder_path : str
        String with the path where the pkl files are saved i.e.,
        '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/02_Clotting_Labelling'

    Returns
    -------
    pickles : dict
        Dictionary since from the previous notebook 02_Clotting_Labelling data were saved in this format
    '''
    # List all files in the directory (assuming all are .pkl files)
    all_files = os.listdir(folder_path)
    
    # Load each .pkl file and store the results in a dictionary
    pickles = {}
    for pkl_file in all_files:
        file_path = os.path.join(folder_path, pkl_file)
        with open(file_path, 'rb') as f:
            pickles[pkl_file] = pickle.load(f)
    
    return pickles

In [3]:
# Load dictionaries 
folder_path = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/02_Clotting_Labelling' 
path_to_save1 = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/03_Combined_dict'
path_to_save2 = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/03_01_Sepatate_dict'
loaded_pickles = load_all_pickles(folder_path)

no_clotting_dict = {}
clotting_dict = {}

# Iterate through the original dictionary and sort based on the key
for key, value in loaded_pickles.items():
    if "no_clotting" in key:
        no_clotting_dict[key] = value
    elif "clotting" in key:
        clotting_dict[key] = value


Now I want to run the quality control. I need to iterate through each .pkl file. Each file is composed of multiple time series; I want to discard those whose len(time series) < 40. I would like to print the len of each dictionary after running this test    

In [4]:
# Function to remove DataFrames with length < 70
def remove_small_dfs(outer_dict, min_length):
    '''The function returns only the Data Frames in which the length is >= 70
    Parameters
    ----------
    outer_dict : dict 
        Dictionary of dictionaries. See the structure below
            data_dict = {
                        'inner_dict_1': {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                        },
                        'inner_dict_2': {
                            'df_trt_3': df3
                        }
                    }
        Each one of the inner dictionaries contains a set of Data Frames, each Data Frame corresponds to a single treatment 

    Returns
    -------
    outer_dict : dict
        Dictionary with the same structure as the input but without those Data Frames whose length < min_length  
    '''
    for key in outer_dict:
        inner_dict = outer_dict[key]
        outer_dict[key] = {df_name: df for df_name, df in inner_dict.items() if len(df) >= min_length}


def remove_undesired_columns(outer_dict,columns):
    '''The function returns a new version of Data Frames in which we preserve only the columns of interest 
    Parameters
    ----------
    outer_dict : dict 
        Dictionary of dictionaries. See the structure below
            data_dict = {
                        'inner_dict_1': {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                        },
                        'inner_dict_2': {
                            'df_trt_3': df3
                        }
                    }
        Each one of the inner dictionaries contains a set of Data Frames, each Data Frame corresponds to a single treatment 

     columns : list
        List with the columns we want to remove
        
    Returns
    -------
    outer_dict : dict
        Dictionary with the same structure as the input but in which each Data Frame has only the columns of interest
        
    '''    
    for key in outer_dict:
        inner_dict = outer_dict[key]
        outer_dict[key] = {df_name: df.drop(columns=columns, errors='ignore') 
                           for df_name, df in inner_dict.items()
                          }


def remove_remaining_data(outer_dict):
    '''The function returns a new version of Data Frames, only in the case of blocking/clotting, in which we cut all the time series data
    after detecting the blocking/clotting event 
    Parameters
    ----------
    outer_dict : dict 
        Dictionary of dictionaries. See the structure below
            data_dict = {
                        'inner_dict_1': {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                        },
                        'inner_dict_2': {
                            'df_trt_3': df3
                        }
                    }
        Each one of the inner dictionaries contains a set of Data Frames, each Data Frame corresponds to a single treatment         
  
    Returns
    -------
    outer_dict : dict
        Dictionary with the same structure as the input but in which each Data Frame, from blocking/clotting, has cut all the time series data
    after detecting the blocking/clotting event 
    '''
    for key in outer_dict:
        inner_dict = outer_dict[key]
        outer_dict[key] = {df_name: df[df['Clotting_2'].ne(df['Clotting_2'].shift()).cumsum() <= 2]
                           for df_name, df in inner_dict.items()
                          }

def length_total(dict_primal):
    '''The function returns the length of the seconday dictionaries embedded on a primal dictionary 
    Parameters
    ----------
    dict_primal : dict 
        Dictionary of dictionaries. See the structure below
            data_dict = {
                        'inner_dict_1': {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                        },
                        'inner_dict_2': {
                            'df_trt_3': df3
                        }
                    }
        Each one of the inner dictionaries contains a set of Data Frames, each Data Frame corresponds to a single treatment 

    Returns
    -------
    None : 
        
    '''
    for keys in list(dict_primal.keys()):
        print(f'{keys} {len(dict_primal[keys].items())}')

    
def combined_items(dict_primal):
    '''The function returns a dictionary combining the Data Frames corresponding to the seconday dictionaries 
    Parameters
    ----------
    dict_primal : dict 
        Dictionary of dictionaries. See the structure below
            dict_primal = {
                        'inner_dict_1': {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                        },
                        'inner_dict_2': {
                            'df_trt_3': df3
                        }
                    }
        Each one of the inner dictionaries contains a set of Data Frames, each Data Frame corresponds to a single treatment 

    Returns
    -------
    combined_dict : dict
        Combined dictionary from dict_primal. See the structure below
            combined_dict = {
                            'df_trt_1': df1,
                            'df_trt_2': df2
                            'df_trt_3': df3
                        }
    '''
    combined_dict = {}
    for keys in list(dict_primal.keys()):
        for name, df in dict_primal[keys].items():
            combined_dict[name] = df
    return combined_dict



In [5]:
def remove_undesired_data_custom(outer_dict, shift):
    for key in outer_dict:
        inner_dict = outer_dict[key]
        outer_dict[key] = {df_name: df.iloc[:df[df['Clotting_2'] == 1].index[0] + shift]
                           for df_name, df in inner_dict.items()
                          }

In [6]:
columns=['Date__Heure', 'trt', 'Patient_weight__Kg_' , 'Set', 'Condition_1', 'Condition_2', 'Delta_P_ref', 'TMP_ref', 'Clotting_1', 'group']
min_length = 70
shift = 15  # number of points we want to conserve once we identify a blocking event 
# Filter for removing small Data Frames (length)
remove_small_dfs(no_clotting_dict, min_length)
remove_small_dfs(clotting_dict, min_length)

#Filter for removing undesired columns
remove_undesired_columns(no_clotting_dict,columns)
remove_undesired_columns(clotting_dict,columns)

#Filter for cutting, in the case of blocking data, the elements after the blocking event
remove_remaining_data(clotting_dict)
#remove_undesired_data_custom(clotting_dict, shift)

<div class="alert alert-block alert-info"> <b>NOTE regarding remove_undesired_data_custom() function.</b> With the results obtained so far, it seems that this function is obsolete since the model performs better when we add as many blocking points as possible. This is easily done with the function remove_undesired_columns(). </div>

Lets see how many data frames from each class do we have

In [7]:
length_total(clotting_dict)

completo1007_(edit)_clotting.pkl 18
completo600_(edit)_clotting.pkl 15
completo400_(edit)_clotting.pkl 19
completo_800_output_file_clotting.pkl 25
completo200_(edit)_clotting.pkl 14


In [8]:
length_total(no_clotting_dict)

completo400_(edit)_no_clotting.pkl 121
completo600_(edit)_no_clotting.pkl 155
completo1007_(edit)_no_clotting.pkl 157
completo200_(edit)_no_clotting.pkl 141
completo_800_output_file_no_clotting.pkl 131


In [11]:
no_clotting_dict['completo1007_(edit)_no_clotting.pkl'].keys()

dict_keys(['df_801', 'df_802', 'df_806', 'df_807', 'df_808', 'df_809', 'df_810', 'df_811', 'df_812', 'df_813', 'df_816', 'df_817', 'df_818', 'df_819', 'df_820', 'df_821', 'df_822', 'df_823', 'df_824', 'df_826', 'df_827', 'df_828', 'df_829', 'df_830', 'df_832', 'df_833', 'df_835', 'df_836', 'df_839', 'df_840', 'df_842', 'df_843', 'df_844', 'df_845', 'df_846', 'df_849', 'df_850', 'df_851', 'df_853', 'df_854', 'df_855', 'df_856', 'df_857', 'df_858', 'df_860', 'df_861', 'df_862', 'df_863', 'df_868', 'df_869', 'df_870', 'df_871', 'df_872', 'df_874', 'df_875', 'df_876', 'df_877', 'df_878', 'df_879', 'df_880', 'df_881', 'df_882', 'df_883', 'df_884', 'df_885', 'df_886', 'df_887', 'df_888', 'df_889', 'df_891', 'df_894', 'df_895', 'df_896', 'df_897', 'df_898', 'df_899', 'df_900', 'df_901', 'df_903', 'df_904', 'df_905', 'df_906', 'df_907', 'df_908', 'df_910', 'df_914', 'df_918', 'df_920', 'df_921', 'df_924', 'df_925', 'df_926', 'df_928', 'df_929', 'df_930', 'df_933', 'df_935', 'df_936', 'df_937',

In [15]:
clotting_dict.keys()
no_clotting_dict['completo1007_(edit)_no_clotting.pkl']['df_802']['Clotting_2'].sum()

0

In [16]:
#Combine all the data into a single random shuffle dictionary 
random.seed(42)
global_dict = {**combined_items(clotting_dict), **combined_items(no_clotting_dict)}
items = list(global_dict.items())
random.shuffle(items)
global_dict = dict(items)

In [17]:
len(global_dict)

796

In [18]:
# Save combined dictionary 
name_clot = f'{path_to_save1}/blocking_no_blocking_shift_15.pkl' 
with open(name_clot, 'wb') as file:
    pickle.dump(global_dict, file)